In [4]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import scale
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [5]:
def convert_SR(data, normal_first=False, SR=True, normal_last=False):
    if normal_first:
        data1 = scale(data)
    else:
        data1 = data.copy()

    if SR:
        A = np.fft.fft(data1)
        L = np.log(abs(A))
        P = np.angle(A)
        h = np.ones((7,),np.float32)/7
        A_L = np.convolve(L, h, 'same')
        R = L - A_L
        S = np.square(abs(np.fft.ifft(np.exp(R + 1j*P))))
    else:
        S = data1.copy()
    
    if normal_last:
        data_out = scale(S)
    else:
        data_out = S.copy()

    return data_out



In [6]:
def save_files(file_in, suffix, normal_first=False, SR=False, normal_last=False):
    data = pd.read_csv('../../data/labelled-skyspark-data/' + file_in + '.csv', parse_dates = ['Datetime'])
    data_trans = convert_SR(data.Value, normal_first=normal_first, SR=normal_first, normal_last=normal_first)
    data_out = data.copy()
    data_out['Value'] = data_trans
    data_out.to_csv(file_in + '_' + suffix + '.csv', index = False)

In [11]:
def plot_various(file_in, plot_portion = 'all'):
    data = pd.read_csv('../../data/labelled-skyspark-data/' + file_in + '.csv', parse_dates = ['Datetime'])

    trace1 = convert_SR(data.Value, normal_first=True, SR=False, normal_last=False)
    trace2 = convert_SR(data.Value, normal_first=False, SR=True, normal_last=False)
    trace3 = convert_SR(data.Value, normal_first=True, SR=True, normal_last=False)
    trace4 = convert_SR(data.Value, normal_first=False, SR=True, normal_last=True)

    if plot_portion == 'first_half':
        start = 0
        end = int(len(data)/2)

    elif plot_portion == 'second_half':
        start = int(len(data)/2)
        end = len(data)
    else:
        start = 0
        end = len(data)

    fig = make_subplots(rows=1, cols=4, subplot_titles=("Standardize, No SR", 
    "No Standardize, SR", 
    "Standardize, then SR", 
    "SR, then Standardize"))

    fig.append_trace(go.Scattergl(
        x = data['Datetime'][start:end],
        y = trace1[start:end],
        mode = 'markers',
        marker=dict(color=1*data['Anomaly'][start:end])
    ), row=1, col=1)

    fig.append_trace(go.Scattergl(
        x = data['Datetime'][start:end],
        y = trace2[start:end],
        mode = 'markers',
        marker=dict(color=1*data['Anomaly'][start:end])
    ), row=1, col=2)

    fig.append_trace(go.Scattergl(
        x = data['Datetime'][start:end],
        y = trace3[start:end],
        mode = 'markers',
        marker=dict(color=1*data['Anomaly'][start:end])
    ), row=1, col=3)

    fig.append_trace(go.Scattergl(
        x = data['Datetime'][start:end],
        y = trace4[start:end],
        mode = 'markers',
        marker=dict(color=1*data['Anomaly'][start:end])
    ), row=1, col=4)

    fig.update_layout(height=500, width=1200,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=2
    ))
    fig.show()

In [30]:
#plot_various('CEC_compiled_data_1b_updated')

![](images/CEC_compiled_data_1b_updated.png)

In [34]:
#plot_various('CEC_compiled_data_2b_updated')

![](images/CEC_compiled_data_2b_updated.png)

In [13]:
#plot_various('CEC_compiled_data_3b_updated', plot_portion='first_half')

![](images/CEC_compiled_data_3b_updated_first.png)

In [15]:
#plot_various('CEC_compiled_data_3b_updated', plot_portion='second_half')

![](images/CEC_compiled_data_3b_updated_second.png)

In [31]:
#plot_various('CEC_compiled_data_4b_updated')

![](images/CEC_compiled_data_4b_updated.png)

In [32]:
#plot_various('CEC_compiled_data_5b_updated')

![](images/CEC_compiled_data_5b_updated.png)

In [ ]:
# run this cell to save various options

# normalize, then SR
save_files('CEC_compiled_data_1b_updated', '_NormSR', normal_first=True, SR=True, normal_last=False)
save_files('CEC_compiled_data_2b_updated', '_NormSR', normal_first=True, SR=True, normal_last=False)
save_files('CEC_compiled_data_3b_updated', '_NormSR', normal_first=True, SR=True, normal_last=False)
save_files('CEC_compiled_data_4b_updated', '_NormSR', normal_first=True, SR=True, normal_last=False)
save_files('CEC_compiled_data_5b_updated', '_NormSR', normal_first=True, SR=True, normal_last=False)
